In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys
from collections import defaultdict

from scipy.optimize import curve_fit

import glob
from matplotlib.backends import backend_pdf

import warnings
warnings.filterwarnings('ignore')

import sqlite3

def readSqlitedb(database="/cvmfs/icarus.opensciencegrid.org/products/icarus/icarus_data/v09_79_01/icarus_data/database/ChannelMapICARUS_20230829.db", table="pmt_placements_29aug2023"):

    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(database)
    df = pd.read_sql_query("SELECT * from {}".format(table), con)

    con.close()

    return df


# grabbing the latest db with the correct channel ID to PMT ID mapping
def load_hv(filename, voltages):

    """
    Makes a dictionary with key the channelId and as value the voltage set
    """
    geo = readSqlitedb()

    for line in open(filename, "r"):

        buff = line.split(",")

        if "{icarus" in buff[0]:

            try: 
                pmtid = int(buff[6])
                channel_id = geo[geo.pmt_id==pmtid].channel_id.values[0]
                value = int( buff[7] )
                          
                voltages[channel_id].append(value)
            
            except ValueError:
                continue 
    return

def load_files(runhvdb, path="/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/equalization"):
    
    files = [ glob.glob(path+'_run%d_*.csv'%run)[0] for run in runhvdb.keys() ]
    df = pd.concat( [pd.read_csv(file, sep=',') for file in files ] )
    df=df.groupby("pmt").agg({'q':list, 'eq':list}).reset_index()
    
    return df



def get_list_for_ch( ch, df, voltages):
    
    bkg_q = np.array(df[df.pmt==ch].q.values[0])
    bkg_eq = np.array(df[df.pmt==ch].q.values[0])*0.02
    
    #filters for null or 0 values
    g = bkg_q[np.where( bkg_q > 0.05 )]
    eg = bkg_eq[np.where( bkg_q > 0.05 )]
    v = np.array(voltages[ch])[np.where( bkg_q > 0.05 )]
    
    return v, g, eg

def poly( v, a, k ):
    return a*np.power(v,k)

def poly_derivative ( v, a, k):
    return a*k*np.power(v, k-1)

def ipoly( g, a, ea, k, ek ): 
    
    v0 = np.power(g/a, 1./k)
    
    dvda = -(1./k)*np.power(0.5, 1./k)*np.power(1./a, 1./(k+1))
    dvdk = -(1./k**2)*np.power(0.5, 1./k)*np.power( (1./a)*(np.log(1./a)-np.log(0.5)), 1./k )
    
    ev0 = np.sqrt( (dvda*ea)**2 + (dvdk*ek)**2 )
    
    return v0, ev0

def dofit( data_x, data_y, error_y, g_target=0.4 ):
    """
    Args: 
        g_target: target gain in units of 10^7
    Returns: 
        ..
        ..
    """
    
    btrials=[5,6,7,7.5,8]
    trials=0
    b=btrials[0]
    
    params=[]
    pcov=[]
    
    repeatfit=True
    
    while repeatfit:
        try:
            #print("Try fit using b: %.2f" % b)
            params, pcov = curve_fit(poly, data_x, data_y, sigma=error_y, absolute_sigma=True, p0=[1e-13, b])
            repeatfit=False
        except:
            trials=trials+1
            if trials<len(btrials):
                b=btrials[trials]
                #print("Repeat fit using b: %.2f" % b)
            else: 
                #print("Finished the amount of trials available for fit")
                raise 
        else: 
            repeatfit=False
            
            
    stderr = np.sqrt(np.diag(pcov))
    
    xint = np.linspace( np.min(data_x)-10, np.max(data_x)+10, 1000 )
    yint = poly(xint, params[0], params[1])
    
    vnom, evnom= ipoly( g_target, params[0], stderr[0] , params[1], stderr[1] )
    
    #print(params, np.sqrt(np.diag(pcov)), vnom, evnom)
    
    return params, np.sqrt(np.diag(pcov)), xint, yint, vnom, evnom



def plotone( PMT, bb, vv ):
    fig, ax = plt.subplots(1,1, figsize=(10, 4.8))

    v, g, eg = get_list_for_ch(PMT, bb, vv)
    print( v, g)

    
    ax.set_ylabel("Gain [$10^7$ electrons]")
    ax.set_xlabel("Voltage [V]")
    ax.legend(fontsize=16)
    
    kargs={'marker':'o', 'lw':0.0, 'elinewidth':2.0}

    out=ax.errorbar( x=v, y=g, yerr=eg, label="Channel ID: %d" % PMT, **kargs )
    res = dofit( v, g, eg )

    p=res[0]
    ep=res[1]
    xint=res[2]
    yint=res[3]
    vnom=res[4]
    evnom=res[5] 

    label= "FIT $a \cdot V^k$: \n"
    label+="a: %.2e $\pm$ %3.e\n" % ( p[0], ep[0] ) 
    label+="b: %.2f $\pm$ %.3f\n" % ( p[1], ep[1] )
    label+='V($4.0 \cdot 10^6$)= %d $\pm$ %d' % ( vnom, evnom )

    ax.plot(xint, yint, color=out[0].get_color(), lw=1.0, label=label)

    ax.set_ylabel("Gain [$10^7$ electrons]")
    ax.set_xlabel("Voltage [V]")
    ax.legend(fontsize=16)
    
    fig.tight_layout()

    return fig, p[0], p[1]
    

def loaddata(runhvdb):

    bkg = load_files(runhvdb, path="/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/equalization")

    voltages = defaultdict(list)
    for run, suffix in runhvdb.items(): 
        load_hv( "../hv_files/Sy4527channels_{}.sub".format(suffix), voltages )
    return voltages, bkg


In [ ]:
runhvdb = { 10901:"20231017_EASTcalibrated_m30",  10898:"20231017_EASTcalibrated", 10900:"20231017_EASTcalibrated_p30",
            10336:"20230818_WESTonly_m100", 10335:"20230818_WESTonly_m50", 10334: "20230818_WESTonly_m30", 10313:"20230818_nominal", 10333:"20230818_WESTonly_p30" 
          }
voltages, bkg = loaddata(runhvdb)

c = 298
fig, a, b, = plotone(c, bkg, voltages ) 

plt.grid(alpha=0.5,linestyle="dashed")
plt.savefig("ch{}_calib_curv.png".format(c),dpi=100)
plt.show()

In [ ]:
# ONLY EAST
geo = readSqlitedb()

runhvdb = {  10901:"20231017_EASTcalibrated_m30", 10896:"20231016_EASTonly_m100",  10898:"20231017_EASTcalibrated", 10900:"20231017_EASTcalibrated_p30", 10895:"20231016_EASTonly_m50", 
             10894: "20231016_EASTonly_m30", 10890:"20231016_nominal", 10893:"20231016_EASTonly_p30",
          }

voltages, bkg = loaddata(runhvdb)

new_EAST_voltages = {}

pdf = backend_pdf.PdfPages("calibration_EAST_18Oct2023.pdf")

for channel_id in range(0, 360):
    
    pmt_id = geo[geo.channel_id==channel_id].pmt_id.values[0]
    
    ## We skip switched-off channels
    ## 350, 248, 215, 190, 161, 139, 127, 103, 131, 59, 52, 21, 5, 71 
    if channel_id in [350, 248, 215, 190, 161, 139, 127, 103, 131, 59, 52, 21, 5, 71]:
        print("Channel_id: {} , pmt_id: {} is OFF".format(channel_id, pmt_id))    
        new_EAST_voltages[pmt_id] = 0
        continue
    
    # Define the plot
    fig, ax = plt.subplots(1,1, figsize=(10, 4.8))
    kargs={'marker':'o', 'lw':0.0, 'elinewidth':2.0}

    #Fetch data
    v, g, eg = get_list_for_ch(channel_id, bkg, voltages)
    
    ## We skip WEST cryostat
    if channel_id in range(180,360):
        #print("Channel_id: {} , pmt_id: {} is in EAST cryostat!".format(channel_id, pmt_id))
        plt.close()
        continue 
    
    # We skip some problematic channels
    # 51, 79 --> drift issues, would go above 2000V     
    #if channel_id in [ 51, 79 ]:
    #    print("Channel_id: {} , pmt_id: {} is skipped, no voltage change".format(channel_id, pmt_id))    
    #    new_voltages[pmt_id] = v[3] # old nominal
    #    plt.close()
    #    continue 
        
    try:
        out=ax.errorbar( x=v, y=g, yerr=eg, label=("Channel ID: %d" % channel_id), **kargs )
    
        try:
            res = dofit( v, g, eg, g_target=0.4 )
            
            p=res[0]
            ep=res[1]
            xint=res[2]
            yint=res[3]
            vnom=res[4]
            evnom=res[5] 

            label= "FIT $a \cdot V^k$: \n"
            label+="a: %.2e $\pm$ %3.e\n" % ( p[0], ep[0] ) 
            label+="b: %.2f $\pm$ %.3f\n" % ( p[1], ep[1] )
            label+='V($4.0 \cdot 10^6$)= %d $\pm$ %d' % ( vnom, evnom )
            
            ax.plot(xint, yint, color=out[0].get_color(), lw=1.0, label=label)
            
            new_EAST_voltages[pmt_id] = int(vnom)
            
            if vnom > v[-1]:
                print("V {} is larger than {}. Interpolated value for channel_id: {} , pmt_id: {}".format(vnom, v[-1], channel_id, pmt_id))  
            if vnom < v[0]:
                print("V {} is smaller than {}. Interpolated value for channel_id: {} , pmt_id: {}".format(vnom, v[0], channel_id, pmt_id)) 
        
            if vnom > 2000: 
                print("Has V larger than 2000V channel_id: {} , pmt_id: {}. Set it to 0 instead".format(channel_id, pmt_id))      
                new_EAST_voltages[pmt_id] = 0
        
        except:
            print("Impossible to fit channel_id: {} , pmt_id: {}".format(channel_id, pmt_id))   
    except:
        print("Impossible to plot channel_id: {} , pmt_id: {}".format(channel_id, pmt_id)) 
    

    ax.set_ylabel("Gain [$10^7$ electrons]")
    ax.set_xlabel("Voltage [V]")
    ax.legend(fontsize=16)
    
    fig.tight_layout()
    plt.grid(alpha=0.5,linestyle="dashed")
    pdf.savefig( fig )
    
    plt.close()


pdf.close()
print("ALL DONE")

In [ ]:
# ONLY WEST
geo = readSqlitedb()

runhvdb = {  10901:"20231017_EASTcalibrated_m30", 10336:"20230818_WESTonly_m100",  10898:"20231017_EASTcalibrated", 10900:"20231017_EASTcalibrated_p30",
             10335:"20230818_WESTonly_m50", 10334: "20230818_WESTonly_m30", 10313:"20230818_nominal", 10333:"20230818_WESTonly_p30",
          }

voltages, bkg = loaddata(runhvdb)

new_WEST_voltages = {}

pdf = backend_pdf.PdfPages("calibration_WEST_18Oct2023.pdf")

for channel_id in range(0, 360):
    
    pmt_id = geo[geo.channel_id==channel_id].pmt_id.values[0]
    
    ## We skip switched-off channels
    ## 350, 248, 215, 190, 161, 139, 127, 103, 131, 59, 52, 21, 5, 71 
    if channel_id in [350, 248, 215, 190, 161, 139, 127, 103, 131, 59, 52, 21, 5, 71]:
        print("Channel_id: {} , pmt_id: {} is OFF".format(channel_id, pmt_id))    
        new_WEST_voltages[pmt_id] = 0
        continue
    
    # Define the plot
    fig, ax = plt.subplots(1,1, figsize=(10, 4.8))
    kargs={'marker':'o', 'lw':0.0, 'elinewidth':2.0}

    #Fetch data
    v, g, eg = get_list_for_ch(channel_id, bkg, voltages)
    
    ## We skip EAST cryostat
    if channel_id in range(0,180):
        #print("Channel_id: {} , pmt_id: {} is in EAST cryostat!".format(channel_id, pmt_id))
        plt.close()
        continue 
    
    # We skip some problematic channels
    # 51, 79 --> drift issues, would go above 2000V     
    #if channel_id in [ 51, 79 ]:
    #    print("Channel_id: {} , pmt_id: {} is skipped, no voltage change".format(channel_id, pmt_id))    
    #    new_voltages[pmt_id] = v[3] # old nominal
    #    plt.close()
    #    continue 
    
    ### MANUAL FIX for channels with HV issues
    ### current list: 66 (ch 298)
    if channel_id == 298:
        new_WEST_voltages[pmt_id] = 1620
        print("Channel {}, PMD id {}: manually setting voltage to {}".format(channel_id,pmt_id,1620))
        plt.close()
        continue
        
    try:
        out=ax.errorbar( x=v, y=g, yerr=eg, label=("Channel ID: %d" % channel_id), **kargs )
    
        try:
            res = dofit( v, g, eg, g_target=0.4 )
            
            p=res[0]
            ep=res[1]
            xint=res[2]
            yint=res[3]
            vnom=res[4]
            evnom=res[5] 

            label= "FIT $a \cdot V^k$: \n"
            label+="a: %.2e $\pm$ %3.e\n" % ( p[0], ep[0] ) 
            label+="b: %.2f $\pm$ %.3f\n" % ( p[1], ep[1] )
            label+='V($4.0 \cdot 10^6$)= %d $\pm$ %d' % ( vnom, evnom )
            
            ax.plot(xint, yint, color=out[0].get_color(), lw=1.0, label=label)
            
            new_WEST_voltages[pmt_id] = int(vnom)
            
            if vnom > v[-1]:
                print("V {} is larger than {}. Interpolated value for channel_id: {} , pmt_id: {}".format(vnom, v[-1], channel_id, pmt_id))  
            if vnom < v[0]:
                print("V {} is smaller than {}. Interpolated value for channel_id: {} , pmt_id: {}".format(vnom, v[0], channel_id, pmt_id)) 
        
            if vnom > 2000: 
                print("Has V larger than 2000V channel_id: {} , pmt_id: {}. Set it to 0 instead".format(channel_id, pmt_id))      
                new_WEST_voltages[pmt_id] = 0
        
        except:
            print("Impossible to fit channel_id: {} , pmt_id: {}".format(channel_id, pmt_id))   
    except:
        print("Impossible to plot channel_id: {} , pmt_id: {}".format(channel_id, pmt_id)) 
    

    ax.set_ylabel("Gain [$10^7$ electrons]")
    ax.set_xlabel("Voltage [V]")
    ax.legend(fontsize=16)
    
    fig.tight_layout()
    plt.grid(alpha=0.5,linestyle="dashed")
    pdf.savefig( fig )
    
    plt.close()


pdf.close()
print("ALL DONE")

In [ ]:
# create the HV file

def writeHVFile( oldfilename, newfilename, newEASTvoltages, newWESTvoltages, capvalues=False ):

    nfp = open(newfilename, "w")

    for line in open(oldfilename , "r"):

        buff = line.split(",")

        if '{icarus' in buff[0]:

            try:
                pmtID = int(buff[6])
            except ValueError:
                print("Warning: invalid PMT id for {}".format(buff[6]))
                line = ",".join(buff)
                nfp.write(line)
                continue

            oldvalue = int(buff[7])
            value=oldvalue
            
            #If a new value exists override it
            try:
                if pmtID in range(0,180): # WEST
                    value = newWESTvoltages[pmtID]
                elif pmtID in range(180,360): # EAST
                    value = newEASTvoltages[pmtID]
            except:
                print( "WARNING: No voltage changes for PMT {}".format(pmtID) )
                #continue
                

            hwarning = 20
            hcaring = 5

            # Cap values over 2000 V
            if value > 2000:
                print( "WARNING: Capped voltage to 2000V for PMT {}".format(pmtID) )
                value=2000
            #else:
                #print( "Replacing {}V with {}V for PMT {}".format(oldvalue, value, pmtID) )

            # here we write the new line to file
            newline = buff
    
            newline[7] = " " + str( value ) #Main nominal value
            newline[9] = " " + str( value+hwarning )
            newline[10] = " " + str( value+hcaring )
            newline[11] = " " + str( value-hcaring )
            newline[12] = " " + str( value-hwarning )+" }\n"

            line =  ",".join(newline)
            nfp.write(line)

        else:
            line = ",".join(buff)
            nfp.write(line)

    print( "Create new file {}".format(newfilename) )
    nfp.close()

    return


#vv = [new_voltages[key] for key in new_voltages.keys()]
#print( np.mean(vv), np.std(vv), np.min(vv), np.max(vv) )
#plt.hist( vv, bins=20, range=(1300, 2200) )  
#plt.show()

writeHVFile("../hv_files/Sy4527channels_20231017_EASTcalibrated.sub", "../hv_files/Sy4527channels_20231018_all_calibrated.sub", new_EAST_voltages, new_WEST_voltages, True)

In [ ]:
old_hv_dict = defaultdict(list)
new_hv_dict = defaultdict(list)
load_hv( "../hv_files/Sy4527channels_20231017_EASTcalibrated.sub", old_hv_dict )
load_hv( "../hv_files/Sy4527channels_20231018_all_calibrated.sub", new_hv_dict )

old_hv=np.array([ ii[1][0] for ii in old_hv_dict.items() ])
new_hv=np.array([ ii[1][0] for ii in new_hv_dict.items() ])

fig, ax = plt.subplots(1,2, figsize=(16, 5))

ax[0].hist( old_hv, bins=20, range=(950, 2100), alpha=0.3, label="G=4.5e6 (old) \n Mean: %d \n Std %d" %  ( np.mean(old_hv), np.std(old_hv) ) )
ax[0].hist( new_hv, bins=20, range=(950, 2100), alpha=0.3, label="G=4.0e6 (new) \n Mean: %d \n Std %d" %  ( np.mean(new_hv), np.std(new_hv) ) )

ax[0].set_ylabel("# of PMT")
ax[0].set_xlabel("Voltage [V]")
ax[0].grid(alpha=0.5)
ax[0].legend()

ax[1].hist( new_hv-old_hv, bins=100, range=(-50, 50), label=r"G: $4.0 \times 10^7$", histtype='step', lw=2.0 )
ax[1].set_ylabel("# of PMTs")
ax[1].set_xlabel(" Voltage after - before equalization [V] " )
ax[1].grid(alpha=0.5)
ax[1].legend()

#plt.legend()
plt.savefig("voltage_distribution.png",dpi=100)

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.hist( new_hv-old_hv, bins=100, range=(-20, 20), label=r"G: $4.0 \times 10^7$", histtype='step', lw=2.0 )
plt.ylabel("# of PMTs")
plt.xlabel(" Voltage after - before equalization [V] " )
plt.legend()
plt.grid(alpha=0.5,linestyle="dashed")
plt.xlim((-20,20.))
plt.savefig("voltage_histogram.png", dpi=100)

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.scatter(old_hv_dict.keys(),new_hv-old_hv, marker="o")
plt.axhline(y=0.0, color="red", linestyle='dotted')

newdriftPMTs = [ 298 ]
newchange=[]
newold_hv_drift=[]
newnew_hv_drift=[]
for id in newdriftPMTs:
    newchange.append(new_hv_dict[id][0]-old_hv_dict[id][0])
    newold_hv_drift.append(old_hv_dict[id][0])
    newnew_hv_drift.append(new_hv_dict[id][0])
plt.scatter(newdriftPMTs,newchange, marker="x", color="magenta", label="New HV issues")

driftPMTs = [ 282, 288, 296, 307, 310, 319, 346, 350, 59, 63, 81, 93, 95, 142, 148, 169, 170, 179 ] #channels
change=[]
old_hv_drift=[]
new_hv_drift=[]
for id in driftPMTs:
    change.append(new_hv_dict[id][0]-old_hv_dict[id][0])
    old_hv_drift.append(old_hv_dict[id][0])
    new_hv_drift.append(new_hv_dict[id][0])
    
plt.scatter(driftPMTs,change, marker="x", color="red", label="HV issues- now fixed")

nochangePMTs = [ 279 ]
nochange=[]
for id in nochangePMTs:
    nochange.append(new_hv_dict[id][0]-old_hv_dict[id][0])
plt.scatter(nochangePMTs,nochange, marker="o", color="green", label="Previous manual fit")

#for id in range(80,100):
#    print(id)
#    print(new_hv_dict[id][0]-old_hv_dict[id][0])

plt.xlabel("PMT channel ID")
plt.ylabel("Voltage after-before equalization [V]")
plt.legend()
plt.grid(alpha=0.5,linestyle="dashed")
plt.savefig("delta_voltage.png",dpi=100)

In [ ]:
plt.scatter(old_hv,new_hv, marker="o")
#plt.scatter(old_hv_drift,new_hv_drift, marker="x", color="red", label="WEST HV issues\n- now fixed")

plt.xlim((1250,2100))
plt.ylim((1250,2100))
plt.xlabel("Voltage before equalization [V]")
plt.ylabel("Voltage after equalization [V]")
#plt.legend()
plt.grid(alpha=0.5,linestyle="dashed")
plt.savefig("preafter2D_voltage.png",dpi=100)
plt.show()

In [ ]:
runhvdb = {  10336:"WESTonly_m100", 10335:"WESTonly_m50", 10334: "WESTonly_m30", 10313:"nominal", 10333:"WESTonly_p30" }
voltages, bkg = loaddata(runhvdb)

valid_pmts = [ pmt for pmt in range(0, 360) if pmt not in [ 350, 248, 215, 190, 161, 139, 127, 103, 131, 59, 52, 21, 5, 71, 51, 79 ] ]  
gains_array = [  get_list_for_ch(pmt, bkg, voltages)[1] for pmt in valid_pmts ]
gains_diff = [ np.max(val)-np.min(val) for val in gains_array ]
new_volt_array = [  new_hv_dict.get(pmt)[0] for pmt in range(0, 360) if pmt in valid_pmts ]
delta_volt_array = [  old_hv_dict.get(pmt)[0]-new_hv_dict.get(pmt)[0] for pmt in valid_pmts ]

fig, ax = plt.subplots(1,2, figsize=(10, 4.6))

ax[0].scatter( new_volt_array, gains_diff )
ax[1].scatter( delta_volt_array, gains_diff )